In [6]:
import pandas as pd
import datetime as dt
import seaborn as sns
import os
from pathlib import Path

sns.set_style("darkgrid", {"axes.facecolor": ".9"})
sns.set_context("paper")

current_path = Path(os.getcwd())
root = current_path.parent.absolute()
data_folder = str(root) + '/raw_data/'
out_folder = str(root) + '/treated_data/'

In [5]:

xls_file = pd.ExcelFile(data_folder + "Volatility_Data.xlsx")


spx = pd.read_excel(xls_file,'SPX')
vix = pd.read_excel(xls_file,'VIX')
sx5e = pd.read_excel(xls_file,'SX5E')
vstoxx = pd.read_excel(xls_file,'VSTOXX')

index_n_vol = {'SPX': spx, 'VIX': vix, 'SX5E' : sx5e, 'VSTOXX': vstoxx}

impvol_cols = ['Date','3MTH_IMPVOL_100.0%MNY_DF', '3MTH_IMPVOL_105.0%MNY_DF',
       '3MTH_IMPVOL_110.0%MNY_DF', '3MTH_IMPVOL_90.0%MNY_DF',
       '3MTH_IMPVOL_95.0%MNY_DF', '6MTH_IMPVOL_100.0%MNY_DF',
       '6MTH_IMPVOL_105.0%MNY_DF', '6MTH_IMPVOL_110.0%MNY_DF',
       '6MTH_IMPVOL_90.0%MNY_DF', '6MTH_IMPVOL_95.0%MNY_DF',
       '12MTH_IMPVOL_100.0%MNY_DF', '12MTH_IMPVOL_105.0%MNY_DF',
       '12MTH_IMPVOL_110.0%MNY_DF', '12MTH_IMPVOL_90.0%MNY_DF',
       '12MTH_IMPVOL_95.0%MNY_DF', '24MTH_IMPVOL_100.0%MNY_DF',
       '24MTH_IMPVOL_105.0%MNY_DF', '24MTH_IMPVOL_110.0%MNY_DF',
       '24MTH_IMPVOL_90.0%MNY_DF', '24MTH_IMPVOL_95.0%MNY_DF']

non_impvol_cols = ['Date', 'PX_OPEN', 'PX_HIGH', 'PX_LOW', 'PX_LAST', 'PX_VOLUME',
       'VOLUME_TOTAL_CALL', 'PUT_CALL_VOLUME_RATIO_CUR_DAY',
       'VOLUME_TOTAL_PUT', 'TOT_OPT_VOLUME_CUR_DAY', 'OPEN_INT_TOTAL_CALL',
       'OPEN_INT_TOTAL_PUT', 'VOLATILITY_30D', 'VOLATILITY_260D',
       'VOLATILITY_90D', 'VOLATILITY_60D', 'VOLATILITY_360D', 'VOLATILITY_10D',
       'VOLATILITY_20D', 'VOLATILITY_180D', 'HIST_CALL_IMP_VOL']

In [7]:
index_n_vol.keys()

dict_keys(['SPX', 'VIX', 'SX5E', 'VSTOXX'])

In [8]:

def int_to_datetime(ticker: pd.DataFrame, column: str):
       if column not in ticker.columns:
           return None
       for i in range(len(ticker[column])) :
        if type(ticker.iloc[i,0])==int:
            ticker.iloc[i,0] = pd.Timedelta(ticker.iloc[i,0], unit='d') + dt.datetime(1899, 12, 30)

In [9]:
for key in index_n_vol.keys():
       int_to_datetime(index_n_vol[key],'Date')
       index_n_vol[key].dropna(axis=1, how='all', inplace=True)
       #index_n_vol[key].dropna(axis=1, thresh=int(0.6*len(index_n_vol[key])), inplace=True) ??????
       index_n_vol[key].set_index(['Date'],inplace=True)
       index_n_vol[key].rename(columns = lambda x: x+'_'+key,inplace=True)

In [10]:


merged_volatility_data = pd.merge_asof(index_n_vol["VIX"], index_n_vol['SPX'], on="Date")
merged_volatility_data = pd.merge_asof(merged_volatility_data, index_n_vol['SX5E'], on="Date")
merged_volatility_data = pd.merge_asof(merged_volatility_data, index_n_vol['VSTOXX'], on="Date")


In [11]:
merged_volatility_data

,Date,PX_OPEN_VIX,PX_HIGH_VIX,PX_LOW_VIX,PX_LAST_VIX,VOLUME_TOTAL_CALL_VIX,PUT_CALL_VOLUME_RATIO_CUR_DAY_VIX,VOLUME_TOTAL_PUT_VIX,TOT_OPT_VOLUME_CUR_DAY_VIX,OPEN_INT_TOTAL_CALL_VIX,...,12MTH_IMPVOL_100.0%MNY_DF_VSTOXX,12MTH_IMPVOL_105.0%MNY_DF_VSTOXX,12MTH_IMPVOL_110.0%MNY_DF_VSTOXX,12MTH_IMPVOL_90.0%MNY_DF_VSTOXX,12MTH_IMPVOL_95.0%MNY_DF_VSTOXX,24MTH_IMPVOL_100.0%MNY_DF_VSTOXX,24MTH_IMPVOL_105.0%MNY_DF_VSTOXX,24MTH_IMPVOL_110.0%MNY_DF_VSTOXX,24MTH_IMPVOL_90.0%MNY_DF_VSTOXX,24MTH_IMPVOL_95.0%MNY_DF_VSTOXX
0,2000-01-03,24.36,26.15,23.98,24.21,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2000-01-04,24.94,27.18,24.80,27.01,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2000-01-05,27.98,29.00,25.85,26.41,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2000-01-06,26.68,26.71,24.70,25.73,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2000-01-07,25.14,25.17,21.72,21.72,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5377,2021-05-18,18.89,21.45,18.81,21.34,435445.0,0.8205,357293.0,792738.0,6190495.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5378,2021-05-19,22.46,25.96,21.88,22.18,417537.0,1.2020,501893.0,919430.0,4725251.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5379,2021-05-20,22.33,23.50,20.19,20.67,208977.0,0.8944,186913.0,395890.0,4842985.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5380,2021-05-21,20.42,20.89,19.53,20.15,127120.0,1.3804,175475.0,302595.0,4884139.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
merged_volatility_data.to_excel(out_folder + 'Volatility_data_merged.xlsx', na_rep='#N/A N/A', sheet_name = "volatility")